# Data Preprocessing For Supermarket Sales Prediction
Super Market Sales Forecasting is a data-driven project designed to predict future sales performance using historical sales data, seasonal trends, and customer purchasing patterns.

## Libraries
- numpy and pandas
- matplotlib and seaborn
- Pathlib
- sci-kit learn
- warnings

## Tasks
1. Import Libraries
2. Load Dataset
3. Explore Dataset
4. Sanity Check
    - Remove redundant columns
    - Standardize column names
    - Duplicate check
    - Missing value check
5. Exploratory Data Analysis
    - Univariate Analysis
    - Bivariate Analysis
    - Multivariate Analysis
6. Handling Missing Value
7. Outliers Treatment
8. Feature Engineering
9. Save Clean Data

# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from sklearn.impute import KNNImputer
from sklearn.preprocessing import label_binarize, StandardScaler

import warnings
warnings.filterwarnings("ignore")

# Load Dataset

In [2]:
# reading filepath of raw dataset
train_path = Path.cwd().parent / "data" / "Train.csv"
test_path = Path(r"../data/Test.csv")

In [3]:
# read files
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

## Verify Data Load

In [4]:
print("Train DataFrame Shape:", train_df.shape)
train_df.head()

Train DataFrame Shape: (8523, 12)


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [5]:
print("Test DataFrame Shape:", test_df.shape)
test_df.head()

Test DataFrame Shape: (5681, 11)


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [6]:
# concatenate train and test data for preprocessing
main_df = pd.concat([train_df, test_df], ignore_index=True)

In [7]:
print("Shape of Concatenated DataFrame:", main_df.shape)
main_df.head()

Shape of Concatenated DataFrame: (14204, 12)


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


# Sanity Checks

In [8]:
# quick overview of data
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            14204 non-null  object 
 1   Item_Weight                11765 non-null  float64
 2   Item_Fat_Content           14204 non-null  object 
 3   Item_Visibility            14204 non-null  float64
 4   Item_Type                  14204 non-null  object 
 5   Item_MRP                   14204 non-null  float64
 6   Outlet_Identifier          14204 non-null  object 
 7   Outlet_Establishment_Year  14204 non-null  int64  
 8   Outlet_Size                10188 non-null  object 
 9   Outlet_Location_Type       14204 non-null  object 
 10  Outlet_Type                14204 non-null  object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 1.3+ MB


In [9]:
# dataset description
main_df.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,11765.000000,14204.000000,14204.000000,14204.000000,8523.000000
mean,12.792854,0.065953,141.004977,1997.830681,2181.288914
std,4.652502,0.051459,62.086938,8.371664,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.710000,0.027036,94.012000,1987.000000,834.247400
50%,12.600000,0.054021,142.247000,1999.000000,1794.331000
75%,16.750000,0.094037,185.855600,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [10]:
# columns names
main_df.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales'],
      dtype='object')

In [11]:
# drop redundant columns
main_df.drop(columns=['Item_Identifier', 'Outlet_Identifier'], inplace=True)

main_df.columns

Index(['Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_Type',
       'Item_MRP', 'Outlet_Establishment_Year', 'Outlet_Size',
       'Outlet_Location_Type', 'Outlet_Type', 'Item_Outlet_Sales'],
      dtype='object')

In [12]:
# rename column names
main_df.rename(columns={
    "Item_Weight" : "ItemWeight",
    "Item_Fat_Content" : "FatContent",
    "Item_Visibility" : "Visibility",
    "Item_Type" : "ItemType",
    "Item_MRP" : "MRP",
    "Outlet_Establishment_Year" : "OutletEstablishmentYear",
    "Outlet_Size" : "OutletSize",
    "Outlet_Location_Type" : "LocationType",
    "Outlet_Type" : "OutletType",
    "Item_Outlet_Sales" : "OutletSales"
}, inplace=True)

# remove leading and trailing spaces from column names
main_df.columns = main_df.columns.str.strip()

main_df.columns

Index(['ItemWeight', 'FatContent', 'Visibility', 'ItemType', 'MRP',
       'OutletEstablishmentYear', 'OutletSize', 'LocationType', 'OutletType',
       'OutletSales'],
      dtype='object')

In [13]:
# check duplicates
main_df.duplicated().sum()

0

In [14]:
# check missing values
main_df.isnull().sum()

ItemWeight                 2439
FatContent                    0
Visibility                    0
ItemType                      0
MRP                           0
OutletEstablishmentYear       0
OutletSize                 4016
LocationType                  0
OutletType                    0
OutletSales                5681
dtype: int64

In [15]:
# missing percantage
main_df.isnull().sum() / main_df.shape[0] * 100

ItemWeight                 17.171219
FatContent                  0.000000
Visibility                  0.000000
ItemType                    0.000000
MRP                         0.000000
OutletEstablishmentYear     0.000000
OutletSize                 28.273726
LocationType                0.000000
OutletType                  0.000000
OutletSales                39.995776
dtype: float64

**NOTE:** `OutletSales` is our target variable, and ~40% missing is a serious issue. Because rows with missing `OutletSales` cannot contribute to training. So we can remove rows which will leaves us with ~60% of the data (still likely sufficient for modeling), and ensures our training set has valid labels.

In [16]:
# drop missing rows from targeted variables
main_df = main_df.dropna(subset=['OutletSales'])

## Dataset After Sanity Check

In [17]:
# dataframe shape
main_df.shape

(8523, 10)

In [18]:
# columns
main_df.columns

Index(['ItemWeight', 'FatContent', 'Visibility', 'ItemType', 'MRP',
       'OutletEstablishmentYear', 'OutletSize', 'LocationType', 'OutletType',
       'OutletSales'],
      dtype='object')

In [19]:
# missing value status
main_df.isnull().sum()

ItemWeight                 1463
FatContent                    0
Visibility                    0
ItemType                      0
MRP                           0
OutletEstablishmentYear       0
OutletSize                 2410
LocationType                  0
OutletType                    0
OutletSales                   0
dtype: int64

In [20]:
# dataset preview
main_df.head()

,ItemWeight,FatContent,Visibility,ItemType,MRP,OutletEstablishmentYear,OutletSize,LocationType,OutletType,OutletSales
0,9.30,Low Fat,0.016047,Dairy,249.8092,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,5.92,Regular,0.019278,Soft Drinks,48.2692,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,17.50,Low Fat,0.016760,Meat,141.6180,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,1998,NaN,Tier 3,Grocery Store,732.3800
4,8.93,Low Fat,0.000000,Household,53.8614,1987,High,Tier 3,Supermarket Type1,994.7052
